### LIBRARIES

In [1]:
from tkinter import *
from tkinter import filedialog
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import pandas as pd
import win32com.client

### BROWSING FOR PARENT FILE

In [2]:
def browse():
    global filename
    filename = filedialog.askopenfilename(initialdir = '/',
                                          title = 'Select a file',
                                          filetypes = (('Excel workbooks', '*.xlsx*'),
                                                      ('CSV files', '*.csv*'))
                                         )
    print(filename)

def close():
    window.destroy()

window = Tk()
window.title('File explorer')
window.geometry('300x200')
Label(window, text = 'Browse for the parent file').pack()
Button(window, text = 'Browse', command = browse).pack(pady = 10)
Button(window, text = 'Exit', command = close).pack(pady = 10)
window.mainloop()

C:/Users/DELL/Downloads/testdataset.xlsx


### CREATING BRANCHES

OBTAINING STRING COLUMNS

In [3]:
tbl = pd.read_excel(filename, skiprows = 2)
str_cols = list(tbl.dtypes[tbl.dtypes == 'object'].index)

RESTORING DATE COLUMNS

In [4]:
date_cols = list(tbl.dtypes[tbl.dtypes == 'datetime64[ns]'].index)
for col in date_cols:
    tbl[col] = tbl[col].dt.strftime('%d-%m-%Y') 

OBTAINING UNIQUE VALUES IN COLUMNS

In [5]:
uniq = {}
for col in tbl.columns:
    uniq[col] = tbl[col].unique()

FETCHING CONTACTS

In [6]:
def get_contacts(filename):
    contacts = []
    with open(filename, 'r') as f:
        for cont in f:
            contacts.append(cont.strip())
    return contacts

ADDING CONTACTS

In [7]:
def add_contact(filename, contact):
    with open(filename, 'a') as f:
        f.write(contact + '\n')

OBTAINING FILTERS

In [24]:
frames = []
filters = {} #{frame: [fr1, fr2, ...]}
sel_values = {} #{fr1: [val1, val2, ...]}
cols = {} #{frame: [col1, col2, ...]}
opts = {} #{fr1: col} chosen columns
tos = {} #{frame: [to1, to2, ...]}
ccs = {} #{frame: [cc1, cc2, ...]}
contacts = get_contacts('contacts.txt') + ['Other']
#each fr corresponds to one column of the dataframe

def ui_filter(frame):
    filter_num = len(filters[frame]) + 1
    fr = Frame(frame)
    fr.pack()
    Label(fr, text = 'Filter ' + str(filter_num)).pack()
    v_opt = StringVar(fr, 'Select a column', str(fr) + '_col')
    OptionMenu(fr, v_opt, *cols[frame]).pack(pady = 10)
    Button(fr, text = 'Select this column', command = lambda: select_col(v_opt.get(), fr)).pack()
    Button(frame, text = 'Add filter', name = 'add_filter_button', command = lambda: add_filter(frame)).pack(pady = 10)
    filters[frame].append(fr)
    
def select_col(col, fr):
    opts[fr] = fr.getvar(str(fr) + '_col')
    Label(fr, text = 'Select the desired value(s)').pack(pady = 10)
    for val in uniq[col]:
        v = IntVar(fr, 0, col + val)
        Checkbutton(fr, text = val, variable = v, onvalue = 1, offvalue = 0).pack()
    Button(fr, text = 'Apply filter', command = lambda: apply_filter(col, fr)).pack(pady = 10)

def apply_filter(col, fr):
    sel_values[fr] = []
    for val in uniq[col]:
        if fr.getvar(col + val):
            sel_values[fr].append(val)
    if len(filters[fr.master]) > 1:
        Button(fr, text = 'Remove filter', command = lambda: remove_filter(col, fr)).pack(pady = 10)
    cols[fr.master].remove(col)
    
def remove_filter(col, fr):
    filters[fr.master].remove(fr)
    for i, f in enumerate(filters[fr.master]):
        f.nametowidget('!label').configure(text = 'Filter ' + str(i + 1))
    cols[fr.master].append(col)
    fr.destroy()
    
def add_filter(frame):
    frame.nametowidget('add_filter_button').destroy()
    ui_filter(frame)

def ui_to_cc(frame, to_cc):
    if to_cc == 'to':
        Label(frame, text = 'To:').pack(pady = 10)
    else:
        Label(frame, text = 'Cc:').pack(pady = 10)
    frame_to_cc = Frame(frame)
    frame_to_cc.pack()
    v_to_cc = StringVar(frame, 'Select a contact')
    OptionMenu(frame_to_cc, v_to_cc, *contacts).pack()
    Button(frame_to_cc, text = 'Add recipient', command = lambda: add_to_cc(v_to_cc, frame_to_cc, to_cc), name = to_cc + '_button').pack(pady = 10)    
    
def ui():
    book_num = len(frames) + 1
    frame = Frame(window)
    frame.pack()
    Label(frame, text = 'Book ' + str(book_num)).pack(pady = 10)
    filters[frame] = [] 
    cols[frame] = str_cols
    ui_filter(frame)
    # email list
    tos[frame] = []
    ccs[frame] = []
    ui_to_cc(frame, 'to')
    ui_to_cc(frame, 'cc')
    if book_num > 1:
        Button(frame, text = 'Remove book', command = lambda: remove(frame)).pack(pady = 10)
    Button(window, text = 'Add book', name = 'add_button', command = add).pack(pady = 10)
    frames.append(frame)

def add_to_cc(v_to_cc, frame_to_cc, to_cc):
    v_to_val = v_to_cc.get()
    if v_to_val == 'Other':
        frame_to_cc.nametowidget('!optionmenu').destroy()
        frame_to_cc.nametowidget(to_cc + '_button').destroy()
        v_to_cc.set('Enter the contact email')
        Entry(frame_to_cc, textvariable = v_to_cc).pack()
        Button(frame_to_cc, text = 'Add recipient', command = lambda: add_to_cc(v_to_cc, frame_to_cc, to_cc), name = to_cc + '_button').pack(pady = 10)
    elif to_cc == 'to':
        tos[frame_to_cc.master].append(v_to_val)
        #add_contact('contacts.txt', v_to_val)
    else:
        ccs[frame_to_cc.master].append(v_to_val)
        #add_contact('contacts.txt', v_to_val)

def add():
    window.nametowidget('add_button').destroy()
    ui()
    
def remove(frame):
    frames.remove(frame)
    frame.destroy()
    for i, fr in enumerate(frames):
        fname = str(fr)
        fr.nametowidget('!label').configure(text = 'Book ' + str(i + 1))        
    
root = Tk()
root.title('Filters')
#scroll bar addition
mf = Frame(root)
mf.pack(fill = BOTH, expand = 1)
canv = Canvas(mf)
canv.pack(side = LEFT, fill = BOTH, expand = 1)
scbar = Scrollbar(mf, orient = VERTICAL, command = canv.yview)
scbar.pack(side = RIGHT, fill = Y)
canv.configure(yscrollcommand = scbar.set)
canv.bind('<Configure>', lambda e: canv.configure(scrollregion = canv.bbox('all')))
window = Frame(canv)
canv.create_window((125,0), window = window, anchor = 'nw')
ui()
window.mainloop()

APPLYING THE FILTERS

In [11]:
for i, book in enumerate(frames):
    df_filter = tbl.copy()
    wb = load_workbook(filename = filename)
    ws = wb.active
    ws.delete_rows(1, 1)
    ws.delete_rows(3, len(df_filter))
    for fr in filters[book]:
        filter_col = opts[fr]
        val_list = sel_values[fr]
        df_filter = df_filter.loc[df_filter[filter_col].isin(val_list)]
    rows = df_filter.values.tolist()
    for row in rows:
        ws.append(row)
    wb.save('Book ' + str(i + 1) + '.xlsx')

SENDING MAILS

In [ ]:
for i, frame in enumerate(frames):
    ol = win32com.client.Dispatch("outlook.application") #outlook application should be installed prior
    olmailitem = 0x0 #size of the new email
    newmail = ol.CreateItem(olmailitem)
    newmail.Subject = 'Testing Mail'
    newmail.To = ';'.join(tos[frame])
    newmail.CC = ';'.join(ccs[frame])
    newmail.Body = 'Please find property comparison of this app and provide sign off.'
    newmail.Attachments.Add('Book ' + str(i + 1) + '.xlsx')
    #To display the mail before sending it
    newmail.Display() 
    #newmail.Send()